Build the model 30 times and calculate the accuracy.

In [1]:
%config Completer.use_jedi=False
import tensorflow as tf
from tensorflow import keras
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Flatten,Conv1D,MaxPooling1D,Input,LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.backend import argmax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import RobustScaler
from scipy.stats import mode
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import os
from os import path

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# from http://www.johnvinyard.com/blog/?p=268

import numpy as np
from numpy.lib.stride_tricks import as_strided as ast

def norm_shape(shape):
    '''
    Normalize numpy array shapes so they're always expressed as a tuple,
    even for one-dimensional shapes.
    Parameters
        shape - an int, or a tuple of ints
    Returns
        a shape tuple
    '''
    try:
        i = int(shape)
        return (i,)
    except TypeError:
        # shape was not a number
        pass

    try:
        t = tuple(shape)
        return t
    except TypeError:
        # shape was not iterable
        pass

    raise TypeError('shape must be an int, or a tuple of ints')

def sliding_window(a,ws,ss = None,flatten = True):
    '''
    Return a sliding window over a in any number of dimensions
    Parameters:
        a  - an n-dimensional numpy array
        ws - an int (a is 1D) or tuple (a is 2D or greater) representing the size
             of each dimension of the window
        ss - an int (a is 1D) or tuple (a is 2D or greater) representing the
             amount to slide the window in each dimension. If not specified, it
             defaults to ws.
        flatten - if True, all slices are flattened, otherwise, there is an
                  extra dimension for each dimension of the input.
    Returns
        an array containing each n-dimensional window from a
    '''

    if None is ss:
        # ss was not provided. the windows will not overlap in any direction.
        ss = ws
    ws = norm_shape(ws)
    ss = norm_shape(ss)

    # convert ws, ss, and a.shape to numpy arrays so that we can do math in every
    # dimension at once.
    ws = np.array(ws)
    ss = np.array(ss)
    shape = np.array(a.shape)


    # ensure that ws, ss, and a.shape all have the same number of dimensions
    ls = [len(shape),len(ws),len(ss)]
    if 1 != len(set(ls)):
        raise ValueError(\
        'a.shape, ws and ss must all have the same length. They were %s' % str(ls))

    # ensure that ws is smaller than a in every dimension
    if np.any(ws > shape):
        raise ValueError(\
        'ws cannot be larger than a in any dimension.\
 a.shape was %s and ws was %s' % (str(a.shape),str(ws)))

    # how many slices will there be in each dimension?
    newshape = norm_shape(((shape - ws) // ss) + 1)
    # the shape of the strided array will be the number of slices in each dimension
    # plus the shape of the window (tuple addition)
    newshape += norm_shape(ws)
    # the strides tuple will be the array's strides multiplied by step size, plus
    # the array's strides (tuple addition)
    newstrides = norm_shape(np.array(a.strides) * ss) + a.strides
    strided = ast(a,shape = newshape,strides = newstrides)
    if not flatten:
        return strided

    # Collapse strided so that it has one more dimension than the window.  I.e.,
    # the new array is a flat list of slices.
    meat = len(ws) if ws.shape else 0
    firstdim = (np.product(newshape[:-meat]),) if ws.shape else ()
    dim = firstdim + (newshape[-meat:])
    # remove any dimensions with size 1
    #dim = filter(lambda i : i != 1,dim) #sytan modified here

    return strided.reshape(dim)

In [3]:
# Hardcoded number of sensor channels employed in the gyro X challenge - X,Y,Z
SENSOR_CHANNELS = 3
# Hardcoded number of classes in the gesture recognition problem
NUM_CLASSES = 12

# sampling rate: 50Hz, 100 * 1/50 = 2 seconds
# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 128
# Hardcoded step of the sliding window mechanism employed to segment the data (50% overlapping)
SLIDING_WINDOW_STEP = 64

# current path
cwd = os.getcwd()

In [4]:
def debug_print_count(data, string='NA'):
    unique, counts = np.unique(data, return_counts=True)
    print(string)
    print(dict(zip(unique, counts)))

In [5]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

In [6]:
def load_dataset_group(dataset_type):
    if (dataset_type == 'train'):
        #path = 'https://raw.githubusercontent.com/sytan6419/hapt_test/main/traindata.txt'
        path = '../traindata.txt'
    elif (dataset_type == 'test'):
        #path = 'https://raw.githubusercontent.com/sytan6419/hapt_test/main/testdata.txt'
        path = '../testdata.txt'

    data = load_file(path)
    return data[:,0:SENSOR_CHANNELS], data[:,-1]

In [7]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train')
    print("Train set:" + str(trainX.shape) + str(trainy.shape))
    # load all test
    testX, testy = load_dataset_group('test')
    print("Test set:" + str(testX.shape) + str(testy.shape))

    return trainX, trainy, testX, testy

In [8]:
'''
@data_y: take the mode of the array as the activity label.
'''
def har_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = mode(sliding_window(data_y,ws,ss), axis=1)[0] 
    #data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

In [9]:
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [10]:
# Plot the data
def plot_data(original_data, scaled_data=None):
    # plot both together to compare
    #fig, ax=plt.subplots(1,2)
    fig, ax=plt.subplots(1,2)
    sns.distplot(original_data, ax=ax[0], color='y')
    ax[0].set_title("Original Data")
    if scaled_data:
        sns.distplot(scaled_data, ax=ax[1])
        ax[1].set_title("Scaled data")
    plt.show()

In [11]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy, epochs=200, patience=20):
    _verbose, epochs, batch_size, patience = 1, epochs, 128, patience

    n_timesteps, n_features, n_outputs = SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS, NUM_CLASSES

    #input 1 window 1;
    inputs1 = Input(shape=(n_timesteps,n_features)) #timestep: 100, feature:12
    convs1_1 = Conv1D(filters=8, kernel_size=2, strides=1, padding='same', activation='relu')(inputs1)
    pools1_1 = MaxPooling1D(pool_size=2, strides=1)(convs1_1)
    convs1_2 = Conv1D(filters=18, kernel_size=2, strides=1, padding='same', activation='relu')(pools1_1)
    pools1_2 = MaxPooling1D(pool_size=2, strides=1)(convs1_2)
    convs1_3 = Conv1D(filters=36, kernel_size=2, strides=1, padding='same', activation='relu')(pools1_2)
    pools1_3 = MaxPooling1D(pool_size=2, strides=1)(convs1_3)

    lstm_1 = LSTM(64, return_sequences=False)(pools1_3)
    drops3_1 = Dropout(0.5)(lstm_1)
    BN_1 = BatchNormalization()(drops3_1)
    output = Dense(n_outputs, activation='softmax')(BN_1)
    
    model = Model(inputs=inputs1, outputs=output)
    opt = optimizers.Adam(learning_rate=0.002)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    es = EarlyStopping(monitor='loss', patience=patience, verbose=True)
    cp = ModelCheckpoint(filepath='benchmark_cp', monitor='loss', save_weights_only=True)
    #model.fit([trainX[:-1], trainX[1:]], [trainy[:-1], trainy[1:]], epochs=epochs, batch_size=batch_size, verbose=_verbose, callbacks=[es])
    #model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=_verbose)#, callbacks=[es])
    history = model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=_verbose, callbacks=[cp])
    model.load_weights('benchmark_cp')
    #plot_model(model, to_file='HAPT_CNNLSTM_parallel.png', show_shapes=True, show_layer_names=True)
    return model, history

In [12]:
import os

# run an experiment
def run_experiment(repeats=1, epochs=200, patience=20):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    
    # prev_trainX = trainX
    scaler = RobustScaler()
    #scaler = MinMaxScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.fit_transform(testX)
    
    # Sensor data is segmented using a sliding window mechanism
    trainX, trainy = har_sliding_window(trainX, trainy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
    testX, testy = har_sliding_window(testX, testy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)

    # Data is reshaped
    trainX = trainX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D
    testX = testX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D

    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)

    # process to throw the first row
    trainy = trainy[:,1:13]
    testy = testy[:,1:13]

    testy = np.argmax(testy, axis=1)
    testy = testy + 1

    history_list = []
    for r in range(25):
        model_g, history = evaluate_model(trainX, trainy, testX, testy, epochs=epochs, patience=patience)
        
        y_pred = model_g.predict(testX)
        y_predict = np.argmax(y_pred, axis=1)
        y_predict= y_predict + 1 

        print("Loop {}: accuracy: {:.2f}".format(r+1, accuracy_score(testy, y_predict)*100))
        print(confusion_matrix(testy, y_predict))
        print(classification_report(testy, y_predict))
        print("================================================")
        
        history_list.append(history)

        model_dir = 'benchmark_' + str(r)
        if not path.exists(model_dir):
            os.mkdir(model_dir) 
        model_g.save(model_dir)
    
    return history_list

# run the experiment
history_list2 = run_experiment(epochs=200, patience=20)

Train set:(554294, 3)(554294,)
Test set:(261320, 3)(261320,)
Epoch 1/200
68/68 [==============================] - 1s 14ms/step - loss: 1.3186 - accuracy: 0.5521
Epoch 2/200
68/68 [==============================] - 1s 13ms/step - loss: 0.8473 - accuracy: 0.7202
Epoch 3/200
68/68 [==============================] - 1s 13ms/step - loss: 0.8298 - accuracy: 0.7288
Epoch 4/200
68/68 [==============================] - 1s 12ms/step - loss: 0.6910 - accuracy: 0.7799
Epoch 5/200
68/68 [==============================] - 1s 13ms/step - loss: 0.7102 - accuracy: 0.7816
Epoch 6/200
68/68 [==============================] - 1s 13ms/step - loss: 0.6040 - accuracy: 0.8201
Epoch 7/200
68/68 [==============================] - 1s 14ms/step - loss: 0.6051 - accuracy: 0.8215
Epoch 8/200
68/68 [==============================] - 1s 13ms/step - loss: 0.5510 - accuracy: 0.8362
Epoch 9/200
68/68 [==============================] - 1s 14ms/step - loss: 0.5711 - accuracy: 0.8305
Epoch 10/200
68/68 [===================

68/68 [==============================] - 1s 13ms/step - loss: 0.1081 - accuracy: 0.9570
Epoch 162/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1003 - accuracy: 0.9591
Epoch 163/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1012 - accuracy: 0.9573
Epoch 164/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1395 - accuracy: 0.9511
Epoch 165/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1211 - accuracy: 0.9501
Epoch 166/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1157 - accuracy: 0.9553
Epoch 167/200
68/68 [==============================] - 1s 13ms/step - loss: 0.0956 - accuracy: 0.9598
Epoch 168/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1048 - accuracy: 0.9572
Epoch 169/200
68/68 [==============================] - 1s 14ms/step - loss: 0.0921 - accuracy: 0.9629
Epoch 170/200
68/68 [==============================] - 1s 13ms/step - loss: 0.0992 - accuracy: 0

68/68 [==============================] - 1s 13ms/step - loss: 0.1895 - accuracy: 0.9276
Epoch 100/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1808 - accuracy: 0.9327
Epoch 101/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1559 - accuracy: 0.9375
Epoch 102/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1753 - accuracy: 0.9351
Epoch 103/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1866 - accuracy: 0.9308
Epoch 104/200
68/68 [==============================] - 1s 13ms/step - loss: 0.2014 - accuracy: 0.9232
Epoch 105/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1777 - accuracy: 0.9328
Epoch 106/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1537 - accuracy: 0.9386
Epoch 107/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1803 - accuracy: 0.9328
Epoch 108/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2340 - accuracy: 0

68/68 [==============================] - 1s 14ms/step - loss: 0.3058 - accuracy: 0.8949
Epoch 45/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2867 - accuracy: 0.8991
Epoch 46/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2734 - accuracy: 0.9026
Epoch 47/200
68/68 [==============================] - 1s 13ms/step - loss: 0.2716 - accuracy: 0.9054
Epoch 48/200
68/68 [==============================] - 1s 13ms/step - loss: 0.2701 - accuracy: 0.9055
Epoch 49/200
68/68 [==============================] - 1s 13ms/step - loss: 0.2717 - accuracy: 0.9030
Epoch 50/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2711 - accuracy: 0.9073
Epoch 51/200
68/68 [==============================] - 1s 14ms/step - loss: 0.3216 - accuracy: 0.8958
Epoch 52/200
68/68 [==============================] - 1s 13ms/step - loss: 0.2794 - accuracy: 0.9047
Epoch 53/200
68/68 [==============================] - 1s 13ms/step - loss: 0.2762 - accuracy: 0.9023
Epo

Epoch 125/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1475 - accuracy: 0.9408
Epoch 126/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1403 - accuracy: 0.9442
Epoch 127/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1464 - accuracy: 0.9436
Epoch 128/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1550 - accuracy: 0.9391
Epoch 129/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1564 - accuracy: 0.9383
Epoch 130/200
68/68 [==============================] - 1s 13ms/step - loss: 0.2776 - accuracy: 0.9031
Epoch 131/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1758 - accuracy: 0.9307
Epoch 132/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1586 - accuracy: 0.9359
Epoch 133/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1405 - accuracy: 0.9450
Epoch 134/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1362

...

68/68 [==============================] - 1s 13ms/step - loss: 0.2944 - accuracy: 0.8954
Epoch 27/200
68/68 [==============================] - 1s 13ms/step - loss: 0.3021 - accuracy: 0.8928
Epoch 28/200
68/68 [==============================] - 1s 13ms/step - loss: 0.3351 - accuracy: 0.8823
Epoch 29/200
68/68 [==============================] - 1s 13ms/step - loss: 0.2824 - accuracy: 0.9022
Epoch 30/200
68/68 [==============================] - 1s 12ms/step - loss: 0.2685 - accuracy: 0.9014
Epoch 31/200
68/68 [==============================] - 1s 13ms/step - loss: 0.2866 - accuracy: 0.8983
Epoch 32/200
68/68 [==============================] - 1s 13ms/step - loss: 0.2726 - accuracy: 0.9030
Epoch 33/200
68/68 [==============================] - 1s 13ms/step - loss: 0.2674 - accuracy: 0.8987
Epoch 34/200
68/68 [==============================] - 1s 13ms/step - loss: 0.3067 - accuracy: 0.8916
Epoch 35/200
68/68 [==============================] - 1s 13ms/step - loss: 0.3003 - accuracy: 0.8932
Epo

68/68 [==============================] - 1s 13ms/step - loss: 0.1758 - accuracy: 0.9343
Epoch 108/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1567 - accuracy: 0.9410
Epoch 109/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1478 - accuracy: 0.9463
Epoch 110/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1345 - accuracy: 0.9456
Epoch 111/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1381 - accuracy: 0.9453
Epoch 112/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1454 - accuracy: 0.9469
Epoch 113/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1398 - accuracy: 0.9480
Epoch 114/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1296 - accuracy: 0.9499
Epoch 115/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1273 - accuracy: 0.9509
Epoch 116/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1261 - accuracy: 0

68/68 [==============================] - 1s 14ms/step - loss: 0.2570 - accuracy: 0.9058
Epoch 53/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2618 - accuracy: 0.9018
Epoch 54/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2671 - accuracy: 0.9021
Epoch 55/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2642 - accuracy: 0.9029
Epoch 56/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2812 - accuracy: 0.8999
Epoch 57/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2601 - accuracy: 0.9051
Epoch 58/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2681 - accuracy: 0.9018
Epoch 59/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2425 - accuracy: 0.9104
Epoch 60/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2668 - accuracy: 0.9043
Epoch 61/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2621 - accuracy: 0.9052
Epo

INFO:tensorflow:Assets written to: benchmark_23/assets
Epoch 1/200
68/68 [==============================] - 1s 13ms/step - loss: 1.3947 - accuracy: 0.5235
Epoch 2/200
68/68 [==============================] - 1s 13ms/step - loss: 0.8481 - accuracy: 0.7232
Epoch 3/200
68/68 [==============================] - 1s 13ms/step - loss: 0.7395 - accuracy: 0.7746
Epoch 4/200
68/68 [==============================] - 1s 13ms/step - loss: 0.6305 - accuracy: 0.8079
Epoch 5/200
68/68 [==============================] - 1s 13ms/step - loss: 0.5671 - accuracy: 0.8265
Epoch 6/200
68/68 [==============================] - 1s 13ms/step - loss: 0.6201 - accuracy: 0.8034
Epoch 7/200
68/68 [==============================] - 1s 13ms/step - loss: 0.5417 - accuracy: 0.8358
Epoch 8/200
68/68 [==============================] - 1s 13ms/step - loss: 0.5186 - accuracy: 0.8392
Epoch 9/200
68/68 [==============================] - 1s 12ms/step - loss: 0.5442 - accuracy: 0.8331
Epoch 10/200
68/68 [=========================

68/68 [==============================] - 1s 13ms/step - loss: 0.1102 - accuracy: 0.9559
Epoch 162/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1091 - accuracy: 0.9562
Epoch 163/200
68/68 [==============================] - 1s 13ms/step - loss: 0.0996 - accuracy: 0.9606
Epoch 164/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1039 - accuracy: 0.9587
Epoch 165/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1081 - accuracy: 0.9566
Epoch 166/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1103 - accuracy: 0.9597
Epoch 167/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1027 - accuracy: 0.9599
Epoch 168/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1007 - accuracy: 0.9600
Epoch 169/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1140 - accuracy: 0.9602
Epoch 170/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1279 - accuracy: 0

In [18]:
import os

# run an experiment
def run_experiment(repeats=1, epochs=200, patience=20):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    
    # prev_trainX = trainX
    scaler = RobustScaler()
    #scaler = MinMaxScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.fit_transform(testX)
    
    # Sensor data is segmented using a sliding window mechanism
    trainX, trainy = har_sliding_window(trainX, trainy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
    testX, testy = har_sliding_window(testX, testy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)

    # Data is reshaped
    trainX = trainX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D
    testX = testX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D

    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)

    # process to throw the first row
    trainy = trainy[:,1:13]
    testy = testy[:,1:13]

    testy = np.argmax(testy, axis=1)
    testy = testy + 1

    history_list = []
    for r in range(25,30):
        model_g, history = evaluate_model(trainX, trainy, testX, testy, epochs=epochs, patience=patience)
        
        y_pred = model_g.predict(testX)
        y_predict = np.argmax(y_pred, axis=1)
        y_predict= y_predict + 1 

        print("Loop {}: accuracy: {:.2f}".format(r+1, accuracy_score(testy, y_predict)*100))
        print(confusion_matrix(testy, y_predict))
        print(classification_report(testy, y_predict))
        print("================================================")
        
        history_list.append(history)

        model_dir = 'benchmark_' + str(r)
        if not path.exists(model_dir):
            os.mkdir(model_dir) 
        model_g.save(model_dir)
    
    return history_list

# run the experiment
history_list = run_experiment(epochs=200, patience=20)

Train set:(554294, 3)(554294,)
Test set:(261320, 3)(261320,)
Epoch 1/200
68/68 [==============================] - 1s 14ms/step - loss: 1.3597 - accuracy: 0.5314
Epoch 2/200
68/68 [==============================] - 1s 14ms/step - loss: 0.9130 - accuracy: 0.6780
Epoch 3/200
68/68 [==============================] - 1s 13ms/step - loss: 0.7736 - accuracy: 0.7445
Epoch 4/200
68/68 [==============================] - 1s 13ms/step - loss: 0.6870 - accuracy: 0.7815
Epoch 5/200
68/68 [==============================] - 1s 13ms/step - loss: 0.5833 - accuracy: 0.8148
Epoch 6/200
68/68 [==============================] - 1s 13ms/step - loss: 0.5686 - accuracy: 0.8167
Epoch 7/200
68/68 [==============================] - 1s 14ms/step - loss: 0.5489 - accuracy: 0.8294
Epoch 8/200
68/68 [==============================] - 1s 13ms/step - loss: 0.5207 - accuracy: 0.8436
Epoch 9/200
68/68 [==============================] - 1s 13ms/step - loss: 0.4585 - accuracy: 0.8517
Epoch 10/200
68/68 [===================

68/68 [==============================] - 1s 13ms/step - loss: 0.1198 - accuracy: 0.9502
Epoch 162/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1199 - accuracy: 0.9505
Epoch 163/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1122 - accuracy: 0.9543
Epoch 164/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1143 - accuracy: 0.9544
Epoch 165/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1233 - accuracy: 0.9490
Epoch 166/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1146 - accuracy: 0.9540
Epoch 167/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1250 - accuracy: 0.9513
Epoch 168/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1219 - accuracy: 0.9510
Epoch 169/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1095 - accuracy: 0.9545
Epoch 170/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1142 - accuracy: 0

68/68 [==============================] - 1s 14ms/step - loss: 0.3852 - accuracy: 0.8704
Epoch 27/200
68/68 [==============================] - 1s 13ms/step - loss: 0.4190 - accuracy: 0.8626
Epoch 28/200
68/68 [==============================] - 1s 14ms/step - loss: 0.3382 - accuracy: 0.8839
Epoch 29/200
68/68 [==============================] - 1s 14ms/step - loss: 0.3525 - accuracy: 0.8778
Epoch 30/200
68/68 [==============================] - 1s 14ms/step - loss: 0.3403 - accuracy: 0.8787
Epoch 31/200
68/68 [==============================] - 1s 14ms/step - loss: 0.3310 - accuracy: 0.8819
Epoch 32/200
68/68 [==============================] - 1s 14ms/step - loss: 0.3229 - accuracy: 0.8876
Epoch 33/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2948 - accuracy: 0.8949
Epoch 34/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2892 - accuracy: 0.8935
Epoch 35/200
68/68 [==============================] - 1s 14ms/step - loss: 0.2850 - accuracy: 0.8969
Epo

68/68 [==============================] - 1s 13ms/step - loss: 0.1222 - accuracy: 0.9553
Epoch 188/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1224 - accuracy: 0.9533
Epoch 189/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1001 - accuracy: 0.9609
Epoch 190/200
68/68 [==============================] - 1s 13ms/step - loss: 0.0973 - accuracy: 0.9620
Epoch 191/200
68/68 [==============================] - 1s 13ms/step - loss: 0.0917 - accuracy: 0.9628
Epoch 192/200
68/68 [==============================] - 1s 13ms/step - loss: 0.0932 - accuracy: 0.9628
Epoch 193/200
68/68 [==============================] - 1s 13ms/step - loss: 0.0919 - accuracy: 0.9651
Epoch 194/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1120 - accuracy: 0.9575
Epoch 195/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1114 - accuracy: 0.9577
Epoch 196/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1068 - accuracy: 0

68/68 [==============================] - 1s 14ms/step - loss: 0.1157 - accuracy: 0.9538
Epoch 133/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1902 - accuracy: 0.9357
Epoch 134/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1562 - accuracy: 0.9397
Epoch 135/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1259 - accuracy: 0.9529
Epoch 136/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1182 - accuracy: 0.9518
Epoch 137/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1232 - accuracy: 0.9522
Epoch 138/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1283 - accuracy: 0.9500
Epoch 139/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1283 - accuracy: 0.9487
Epoch 140/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1161 - accuracy: 0.9537
Epoch 141/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1170 - accuracy: 0

INFO:tensorflow:Assets written to: benchmark_27/assets
Epoch 1/200
68/68 [==============================] - 1s 13ms/step - loss: 1.3445 - accuracy: 0.5221
Epoch 2/200
68/68 [==============================] - 1s 13ms/step - loss: 0.8788 - accuracy: 0.6860
Epoch 3/200
68/68 [==============================] - 1s 13ms/step - loss: 0.7606 - accuracy: 0.7442
Epoch 4/200
68/68 [==============================] - 1s 14ms/step - loss: 0.6931 - accuracy: 0.7813
Epoch 5/200
68/68 [==============================] - 1s 13ms/step - loss: 0.6631 - accuracy: 0.7884
Epoch 6/200
68/68 [==============================] - 1s 13ms/step - loss: 0.5837 - accuracy: 0.8115
Epoch 7/200
68/68 [==============================] - 1s 14ms/step - loss: 0.5879 - accuracy: 0.8170
Epoch 8/200
68/68 [==============================] - 1s 14ms/step - loss: 0.5260 - accuracy: 0.8382
Epoch 9/200
68/68 [==============================] - 1s 14ms/step - loss: 0.5339 - accuracy: 0.8263
Epoch 10/200
68/68 [=========================

68/68 [==============================] - 1s 14ms/step - loss: 0.1034 - accuracy: 0.9573
Epoch 162/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1176 - accuracy: 0.9492
Epoch 163/200
68/68 [==============================] - 1s 14ms/step - loss: 0.0991 - accuracy: 0.9583
Epoch 164/200
68/68 [==============================] - 1s 13ms/step - loss: 0.0968 - accuracy: 0.9591
Epoch 165/200
68/68 [==============================] - 1s 13ms/step - loss: 0.0862 - accuracy: 0.9621
Epoch 166/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1029 - accuracy: 0.9589
Epoch 167/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1608 - accuracy: 0.9431
Epoch 168/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1298 - accuracy: 0.9485
Epoch 169/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1161 - accuracy: 0.9530
Epoch 170/200
68/68 [==============================] - 1s 14ms/step - loss: 0.0986 - accuracy: 0

68/68 [==============================] - 1s 13ms/step - loss: 0.3682 - accuracy: 0.8762
Epoch 27/200
68/68 [==============================] - 1s 13ms/step - loss: 0.3536 - accuracy: 0.8805
Epoch 28/200
68/68 [==============================] - 1s 13ms/step - loss: 0.3687 - accuracy: 0.8810
Epoch 29/200
68/68 [==============================] - 1s 14ms/step - loss: 0.3142 - accuracy: 0.8926
Epoch 30/200
68/68 [==============================] - 1s 14ms/step - loss: 0.3727 - accuracy: 0.8740
Epoch 31/200
68/68 [==============================] - 1s 13ms/step - loss: 0.3826 - accuracy: 0.8717
Epoch 32/200
68/68 [==============================] - 1s 13ms/step - loss: 0.3650 - accuracy: 0.8755
Epoch 33/200
68/68 [==============================] - 1s 13ms/step - loss: 0.3274 - accuracy: 0.8830
Epoch 34/200
68/68 [==============================] - 1s 13ms/step - loss: 0.3200 - accuracy: 0.8899
Epoch 35/200
68/68 [==============================] - 1s 12ms/step - loss: 0.3095 - accuracy: 0.8906
Epo

68/68 [==============================] - 1s 12ms/step - loss: 0.1138 - accuracy: 0.9562
Epoch 188/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1274 - accuracy: 0.9507
Epoch 189/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1006 - accuracy: 0.9602
Epoch 190/200
68/68 [==============================] - 1s 14ms/step - loss: 0.1072 - accuracy: 0.9552
Epoch 191/200
68/68 [==============================] - 1s 14ms/step - loss: 0.0951 - accuracy: 0.9575
Epoch 192/200
68/68 [==============================] - 1s 13ms/step - loss: 0.0982 - accuracy: 0.9592
Epoch 193/200
68/68 [==============================] - 1s 13ms/step - loss: 0.0931 - accuracy: 0.9599
Epoch 194/200
68/68 [==============================] - 1s 14ms/step - loss: 0.0878 - accuracy: 0.9629
Epoch 195/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1246 - accuracy: 0.9548
Epoch 196/200
68/68 [==============================] - 1s 13ms/step - loss: 0.1065 - accuracy: 0

In [20]:
for i in range(25):
    file_loss = 'hist_loss' + str(i) + '.txt'
    file_acc = 'hist_acc' + str(i) + '.txt'
    f = open(file_loss, "w")
    for e in history_list2[i].history['loss']:
        f.write(str(e) + '\n')
    f.close()
    f = open(file_acc, "w")
    for e in history_list2[i].history['accuracy']:
        f.write(str(e) + '\n')
    f.close()

In [21]:
for i in range(5):
    file_loss = 'hist_loss' + str(i+25) + '.txt'
    file_acc = 'hist_acc' + str(i+25) + '.txt'
    f = open(file_loss, "w")
    for e in history_list[i].history['loss']:
        f.write(str(e) + '\n')
    f.close()
    f = open(file_acc, "w")
    for e in history_list2[i].history['accuracy']:
        f.write(str(e) + '\n')
    f.close()

In [22]:
n_timesteps, n_features, n_outputs = SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS, NUM_CLASSES

#input 1 window 1;
inputs1 = Input(shape=(n_timesteps,n_features)) #timestep: 100, feature:12
convs1_1 = Conv1D(filters=8, kernel_size=2, strides=1, padding='same', activation='relu')(inputs1)
pools1_1 = MaxPooling1D(pool_size=2, strides=1)(convs1_1)
convs1_2 = Conv1D(filters=18, kernel_size=2, strides=1, padding='same', activation='relu')(pools1_1)
pools1_2 = MaxPooling1D(pool_size=2, strides=1)(convs1_2)
convs1_3 = Conv1D(filters=36, kernel_size=2, strides=1, padding='same', activation='relu')(pools1_2)
pools1_3 = MaxPooling1D(pool_size=2, strides=1)(convs1_3)

lstm_1 = LSTM(64, return_sequences=False)(pools1_3)
drops3_1 = Dropout(0.5)(lstm_1)
BN_1 = BatchNormalization()(drops3_1)
output = Dense(n_outputs, activation='softmax')(BN_1)

pred_model = Model(inputs=inputs1, outputs=output)

In [23]:
trainX, trainy, testX, testy = load_dataset()
scaler = RobustScaler() #check if the scaler if this impact the ReLu. https://mlfromscratch.com/activation-functions-explained/#/
testX = scaler.fit_transform(testX)
testX, testy = har_sliding_window(testX, testy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
testX = testX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D
# one hot encode y
testy = to_categorical(testy)
# process to throw the first row
testy = testy[:,1:13]
testy = np.argmax(testy, axis=1)
testy = testy + 1

f = open('model_accuracies.txt', "w") 
for i in range(0,30):
    model_file = 'benchmark_' + str(i)
    pred_model = tf.keras.models.load_model(model_file)
    
    y_pred = pred_model.predict([testX,testX,testX])
    y_predict = np.argmax(y_pred, axis=1)
    y_predict= y_predict + 1 
    print('Model ' + str(i+1))
    accuracy = accuracy_score(testy, y_predict)
    print(accuracy)
    print(confusion_matrix(testy, y_predict))
    print(classification_report(testy, y_predict))
    f.write(str(accuracy) + '\n')
f.close()

Train set:(554294, 3)(554294,)
Test set:(261320, 3)(261320,)
Model 1
0.8829005389514943
[[515  22  12   0   4   0   0  21   0   0   1   5]
 [ 12 541  11   1   2   0   0   0   0   0   0   1]
 [  1  21 514   0   1   0   0   0   0   0   0   0]
 [  0   0   0 539 120   0   1   2   2   4   0   0]
 [  0   1   0  95 600   1   1   5   0   0   5   0]
 [  0   0   0   0   0 716   0   0   1   1   0   1]
 [  0   0   0   7   4   0  30   3   1   0   4   0]
 [  0   0   0   2   4   0   0  34   0   0   0   0]
 [  0   0   0   2   2   7   0   0  36   0  12   0]
 [  0   0   0   2   2   3   3   0   0  31   0   7]
 [  0   0   0   1   2   5   2   0  23   1  21   0]
 [  2   0   0   1   1   5   0   2   0  12   1  27]]
              precision    recall  f1-score   support

           1       0.97      0.89      0.93       580
           2       0.92      0.95      0.94       568
           3       0.96      0.96      0.96       537
           4       0.83      0.81      0.82       668
           5       0.81     

Model 7
0.8851053405193533
[[519  36  15   0   2   0   0   1   0   1   4   2]
 [  3 550   8   0   0   0   5   1   0   1   0   0]
 [  1  12 523   0   0   1   0   0   0   0   0   0]
 [  0   0   0 586  78   0   2   1   0   1   0   0]
 [  1   4   0 143 551   1   1   3   0   0   4   0]
 [  0   0   0   0   1 717   0   1   0   0   0   0]
 [  0   0   0   8   5   0  31   1   0   1   3   0]
 [  1   0   0   3   3   0   1  30   0   0   0   2]
 [  0   0   0   6   0   8   0   2  36   0   7   0]
 [  0   0   0   8   0   4   1   0   0  26   0   9]
 [  0   0   0   4   2   9   2   0  20   0  18   0]
 [  1   0   0   3   2   5   0   1   0  11   2  26]]
              precision    recall  f1-score   support

           1       0.99      0.89      0.94       580
           2       0.91      0.97      0.94       568
           3       0.96      0.97      0.97       537
           4       0.77      0.88      0.82       668
           5       0.86      0.78      0.82       708
           6       0.96      1.00  

Model 13
0.9034786869181773
[[554   6   6   0   0   0   1   1   0   1   6   5]
 [  0 539  16   0   0   0   2   4   0   0   3   4]
 [  2   7 511   0   0   0   8   2   0   1   3   3]
 [  0   0   0 565  97   0   2   3   0   1   0   0]
 [  0   1   0  73 624   0   0   3   2   0   4   1]
 [  0   0   0   0   2 715   0   0   1   0   1   0]
 [  0   0   0   6   6   0  32   3   0   0   1   1]
 [  1   0   0   3   6   0   0  30   0   0   0   0]
 [  0   0   0   3   1   7   1   1  41   0   5   0]
 [  0   0   0  10   0   5   1   0   0  21   0  11]
 [  0   0   0   2   1   8   3   0  19   0  22   0]
 [  0   0   0   2   0   5   0   2   0   8   0  34]]
              precision    recall  f1-score   support

           1       0.99      0.96      0.97       580
           2       0.97      0.95      0.96       568
           3       0.96      0.95      0.96       537
           4       0.85      0.85      0.85       668
           5       0.85      0.88      0.86       708
           6       0.97      0.99 

Model 19
0.9034786869181773
[[555  21   2   1   0   0   0   0   0   0   0   1]
 [  3 556   9   0   0   0   0   0   0   0   0   0]
 [  2   4 531   0   0   0   0   0   0   0   0   0]
 [  0   0   0 574  86   0   4   4   0   0   0   0]
 [  0   3   0 116 581   1   1   1   0   0   5   0]
 [  0   0   0   0   0 714   0   2   0   1   2   0]
 [  0   0   0  11   4   0  27   1   0   1   4   1]
 [  0   0   0   3   5   0   0  31   0   0   0   1]
 [  0   0   0   4   0   7   2   0  20   0  26   0]
 [  0   0   0   8   0   3   0   0   0  33   0   4]
 [  0   0   0   2   1   5   0   0   7   0  40   0]
 [  0   1   0   2   3   6   1   2   0  10   0  26]]
              precision    recall  f1-score   support

           1       0.99      0.96      0.97       580
           2       0.95      0.98      0.96       568
           3       0.98      0.99      0.98       537
           4       0.80      0.86      0.83       668
           5       0.85      0.82      0.84       708
           6       0.97      0.99 

Model 25
0.8870651641352278
[[534  16  16   0   0   0   1   4   0   0   1   8]
 [  3 540  11   1   0   0   0   6   1   0   1   5]
 [  1   7 527   0   0   0   0   0   0   0   0   2]
 [  0   0   0 542 119   0   2   2   2   0   1   0]
 [  0   5   0 111 576   0   1   6   1   0   8   0]
 [  0   0   0   0   0 716   0   2   0   0   1   0]
 [  0   0   0   8   4   0  28   3   1   2   3   0]
 [  1   0   0   3   4   0   0  32   0   0   0   0]
 [  0   0   0   2   0   9   1   0  31   0  16   0]
 [  0   0   0   5   1   3   0   0   0  34   0   5]
 [  0   0   0   2   1   8   0   0  11   1  32   0]
 [  1   0   0   2   0   3   0   2   0  14   0  29]]
              precision    recall  f1-score   support

           1       0.99      0.92      0.95       580
           2       0.95      0.95      0.95       568
           3       0.95      0.98      0.97       537
           4       0.80      0.81      0.81       668
           5       0.82      0.81      0.82       708
           6       0.97      1.00 